### Download Dataset

In [ ]:
!gdown 1J-gnuvPoanMfM52RTY2IpdHOaVL36L3e
!unzip train.zip
!rm -rf /home/bridge/c/nvijayas/ECE50024/__MACOSX

### Extract Faces

In [2]:
import os, random, csv, pickle, cv2, shutil, dlib
from PIL import Image
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
from ultralytics import YOLO

In [18]:
folder_path = 'train'
image_files = [(os.path.join(folder_path, f), f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
len(image_files)

69540

In [ ]:
# Splitting the list
quarter_length = len(image_files) // 4
q1_imFiles = image_files[:quarter_length]
q2_imFiles = image_files[quarter_length:2*quarter_length]
q3_imFiles = image_files[2*quarter_length:3*quarter_length]
q4_imFiles = image_files[3*quarter_length:]

# Pickling the lists
with open('q1_imFiles.pkl', 'wb') as f:
  pickle.dump(q1_imFiles, f)
with open('q2_imFiles.pkl', 'wb') as f:
  pickle.dump(q2_imFiles, f)
with open('q3_imFiles.pkl', 'wb') as f:
  pickle.dump(q3_imFiles, f)
with open('q4_imFiles.pkl', 'wb') as f:
  pickle.dump(q4_imFiles, f)

In [2]:
def getOGlabels(csv_path, ):
  original_labels = {}
  with open(csv_path, mode='r') as infile:
    reader = csv.reader(infile)
    next(reader, None)
    for row in reader:
      original_labels[row[1]] = row[2]

  return original_labels


# For the face extraction, the following code in this link referenced
# https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81
def process(imFiles, output_folder, new_csv_path, original_labels, pName):

  detector = dlib.get_frontal_face_detector()
  noFace = 0
  multiFace = 0
  noLabel = 0
  someFileErr = 0

  with open(new_csv_path, mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['filename', 'label'])

    for ctr, fileInfo in enumerate(imFiles):

      if (ctr + 1) % 1000 == 0:
        print(f"{pName}: {ctr + 1}")

      filePath, fileName = fileInfo[0], fileInfo[1]
      label = original_labels[fileName]

      if not label:
        noLabel += 1
        continue

      try:
        pil_image = Image.open(filePath).convert('RGB')  # filePath is the path to your image file
        img = np.array(pil_image)
        detected_faces = detector(img, 1)

        if len(detected_faces) == 0:
          noFace += 1
          continue
        elif len(detected_faces) > 1:
          multiFace += 1
          continue
        else:
          face_rect = detected_faces[0]
          
          # Adjusted coordinates with a 2-pixel boundary, and clamped to the image bounds
          left = max(face_rect.left() - 2, 0)
          top = max(face_rect.top() - 2, 0)
          right = min(face_rect.right() + 2, img.shape[1])
          bottom = min(face_rect.bottom() + 2, img.shape[0])

          cropped_img = Image.fromarray(img, 'RGB').crop((left, top, right, bottom))
          cropped_img = cropped_img.resize((64, 64))

          new_filename = f"{fileName.rsplit('.', 1)[0]}.jpg"
          cropped_img.save(os.path.join(output_folder, new_filename))
          writer.writerow([new_filename, label])

      except Exception as e:
        someFileErr += 1
        continue
  print(f"PID {pName}\tNoF {noFace}\tMF {multiFace}\tNL {noLabel}\tSFE {someFileErr}")

In [3]:
with open('q3_imFiles.pkl', 'rb') as file:
    q3_imFiles = pickle.load(file)

with open('q4_imFiles.pkl', 'rb') as file:
    q4_imFiles = pickle.load(file)

In [ ]:
def main():
  # Paths
  output_folder3 = 'train_faces3'
  output_folder4 = 'train_faces4'
  os.makedirs(output_folder3, exist_ok=True)
  os.makedirs(output_folder4, exist_ok=True)
  
  new_csv_path3_1 = 'train_faces3-1.csv'
  new_csv_path3_2 = 'train_faces3-2.csv'
  
  new_csv_path4_1 = 'train_faces4-1.csv'
  new_csv_path4_2 = 'train_faces4-2.csv'

  # Load original labels
  original_csv_path = 'train.csv'
  original_labels = getOGlabels(original_csv_path)
  print(len(original_labels))

  # Split list in half
  middle_index3 = len(q3_imFiles) // 2
  half_3_1 = q3_imFiles[:middle_index3]
  half_3_2 = q3_imFiles[middle_index3:]
  print(len(half_3_1))
  print(len(half_3_2))

  # Split list in half
  middle_index4 = len(q4_imFiles) // 2
  half_4_1 = q4_imFiles[:middle_index4]
  half_4_2 = q4_imFiles[middle_index4:]
  print(len(half_4_1))
  print(len(half_4_2))


  # Create two processes
  p3_1 = multiprocessing.Process(target=process, args=(half_3_1, output_folder3, new_csv_path3_1, original_labels, 'p3_1'))
  p3_2 = multiprocessing.Process(target=process, args=(half_3_2, output_folder3, new_csv_path3_2, original_labels, 'p3_2'))

  p4_1 = multiprocessing.Process(target=process, args=(half_4_1, output_folder4, new_csv_path4_1, original_labels, 'p4_1'))
  p4_2 = multiprocessing.Process(target=process, args=(half_4_2, output_folder4, new_csv_path4_2, original_labels, 'p4_2'))

  # Start the processes
  p3_1.start()
  p3_2.start()

  p4_1.start()
  p4_2.start()

  # Wait for the processes to finish
  p3_1.join()
  p3_2.join()
  
  p4_1.join()
  p4_2.join()

if __name__ == '__main__':
  main()

### Combine all images + csvs

In [3]:
train_faces_dir = '/home/bridge/c/nvijayas/ECE50024/train_faces'
os.makedirs(train_faces_dir, exist_ok=True)

to_combine = ['/home/bridge/c/nvijayas/ECE50024/train_faces1',
              '/home/bridge/c/nvijayas/ECE50024/train_faces2',
              '/home/bridge/c/nvijayas/ECE50024/train_faces3',
              '/home/bridge/c/nvijayas/ECE50024/train_faces4']

# Move all files from each source folder to the destination folder
for folder in to_combine:
  print(folder)
  for filename in os.listdir(folder):
    source_file = os.path.join(folder, filename)
    destination_file = os.path.join(train_faces_dir, filename)
    if not os.path.exists(destination_file):
      shutil.move(source_file, destination_file)
    else:
      print(f"File {filename} already exists.")

/home/bridge/c/nvijayas/ECE50024/train_faces1
/home/bridge/c/nvijayas/ECE50024/train_faces2
/home/bridge/c/nvijayas/ECE50024/train_faces3
/home/bridge/c/nvijayas/ECE50024/train_faces4


In [4]:
len(os.listdir('/home/bridge/c/nvijayas/ECE50024/train_faces'))

52502

In [5]:
import csv
import glob

In [8]:
# Path to the directory containing your CSV files
csv_dir_path = '/home/bridge/c/nvijayas/ECE50024'
# The pattern matches all CSV files in the directory
csv_files = glob.glob(f'{csv_dir_path}/*.csv')
print(csv_files)
csv_files = csv_files[1:]
print(csv_files)
print(len(csv_files))

# The path for the new combined CSV file
combined_csv_path = '/home/bridge/c/nvijayas/ECE50024/train_faces.csv'

['/home/bridge/c/nvijayas/ECE50024/train.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces3-1.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces3-2.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces4-1.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces4-2.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces1-2.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces1-1.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces2-1.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces2-2.csv']
['/home/bridge/c/nvijayas/ECE50024/train_faces3-1.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces3-2.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces4-1.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces4-2.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces1-2.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces1-1.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces2-1.csv', '/home/bridge/c/nvijayas/ECE50024/train_faces2-2.csv']
8


In [9]:
# Initialize a variable to track if the header has been written to the new file
header_written = False

# Initialize a counter for the rows
row_count = 0

with open(combined_csv_path, 'w', newline='') as combined_file:
    writer = csv.writer(combined_file)
    for file in csv_files:
        with open(file, 'r', newline='') as read_file:
            reader = csv.reader(read_file)
            header = next(reader)
            if not header_written:
                writer.writerow(header)  # Write the header in the new file
                header_written = True
            for row in reader:
                writer.writerow(row)  # Write the data rows
                row_count += 1  # Increment the row count

print("CSV files have been combined.")
# Print the total number of data rows written to the new file
print(f"Total rows (excluding the header): {row_count}")


CSV files have been combined.
Total rows (excluding the header): 52502


In [10]:
# Path to your CSV file
csv_file_path = '/home/bridge/c/nvijayas/ECE50024/train_faces.csv'
data_dir = '/home/bridge/c/nvijayas/ECE50024/data4yolo'
root_dir = '/home/bridge/c/nvijayas/ECE50024/train_faces'
os.makedirs(data_dir, exist_ok=True)

# Open and read the CSV file
with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    # Loop through each row in the CSV
    for row in csv_reader:
        filename = row['filename']
        label = row['label']
        mod_label = label.replace(" ", "_")
        dirLabel = os.path.join(data_dir, mod_label)
        
        # Create a directory for the label if it doesn't exist
        if not os.path.exists(dirLabel):
            os.makedirs(dirLabel)
        
        # Define the source and destination paths
        source_path = os.path.join(root_dir, filename)
        destination_path = os.path.join(dirLabel, filename)
        
        # Copy the file to the destination directory
        shutil.move(source_path, destination_path)


In [18]:
def count_subfolders(directory):
    """Count the number of subfolders in the given directory."""
    subfolders = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    return len(subfolders)

# Specify the directory to check
directory_path = '/home/bridge/c/nvijayas/ECE50024/data4yolo/val'

# Call the function and print the result
number_of_subfolders = count_subfolders(directory_path)
print(f'There are {number_of_subfolders} subfolders in {directory_path}')


There are 100 subfolders in /home/bridge/c/nvijayas/ECE50024/data4yolo/val


In [19]:
from math import ceil

In [15]:
def create_validation_set(data_root, valid_ratio=0.2):
    train_dir = os.path.join(data_root, 'train')
    valid_dir = os.path.join(data_root, 'val')
    
    # Check if the validation directory exists, create if not
    if not os.path.exists(valid_dir):
        os.makedirs(valid_dir)
    
    # Iterate over each class directory in the train directory
    for class_name in os.listdir(train_dir):
        class_dir = os.path.join(train_dir, class_name)
        
        # Check if the class directory in the validation set exists, create if not
        valid_class_dir = os.path.join(valid_dir, class_name)
        if not os.path.exists(valid_class_dir):
            os.makedirs(valid_class_dir)
        
        # List all files in the current class directory
        files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        random.shuffle(files)
        
        # Calculate the number of validation samples
        num_valid_samples = ceil(len(files) * valid_ratio)
        
        # Move the files
        for file in files[:num_valid_samples]:
            shutil.move(os.path.join(class_dir, file), os.path.join(valid_class_dir, file))

# Your data root directory
data_root = '/home/bridge/c/nvijayas/ECE50024/data4yolo'
create_validation_set(data_root)


### Train

In [ ]:
# Used pre-trained YOLO model
# Referenced code from Ultraylyics docs 
# https://docs.ultralytics.com

# Load a model
model = YOLO('/home/bridge/c/nvijayas/ECE50024/runs/classify/train4/weights/last.pt')

# Train the model
results = model.train(data='/home/bridge/c/nvijayas/ECE50024/data4yolo', 
                      epochs=50, 
                      imgsz=64, 
                      device= [4, 7], 
                      lr0=0.01, 
                      lrf=0.008)

In [ ]:
# Validate the model
metrics = model.val()
metrics.top1   # top1 accuracy
metrics.top5   # top5 accuracy

### Create Test Faces

In [31]:
def process_test(imFiles, output_folder, pName):
  detector = dlib.get_frontal_face_detector()
  noFace = 0
  multiFace = 0
  noLabel = 0
  someFileErr = 0
  files_not_processed = []

  for ctr, fileInfo in enumerate(imFiles):

    if (ctr + 1) % 1000 == 0:
      print(f"{pName}: {ctr + 1}")

    filePath, fileName = fileInfo[0], fileInfo[1]
  
    try:
      pil_image = Image.open(filePath).convert('RGB')  # filePath is the path to your image file
      img = np.array(pil_image)
      detected_faces = detector(img, 1)

      if len(detected_faces) == 0:
        noFace += 1
        files_not_processed.append(fileInfo)
        continue
      elif len(detected_faces) > 1:
        multiFace += 1
        files_not_processed.append(fileInfo)
        continue
      else:
        face_rect = detected_faces[0]
        
        # Adjusted coordinates with a 2-pixel boundary, and clamped to the image bounds
        left = max(face_rect.left() - 2, 0)
        top = max(face_rect.top() - 2, 0)
        right = min(face_rect.right() + 2, img.shape[1])
        bottom = min(face_rect.bottom() + 2, img.shape[0])

        cropped_img = Image.fromarray(img, 'RGB').crop((left, top, right, bottom))
        cropped_img = cropped_img.resize((64, 64))

        new_filename = f"{fileName.rsplit('.', 1)[0]}.jpg"
        cropped_img.save(os.path.join(output_folder, new_filename))

    except Exception as e:
      someFileErr += 1
      files_not_processed.append(fileInfo)
      continue
  print(f"PID {pName}\tNoF {noFace}\tMF {multiFace}\tNL {noLabel}\tSFE {someFileErr}")
  return files_not_processed

In [33]:
test_dir = '/home/bridge/c/nvijayas/ECE50024/test'
test_imgs = [(os.path.join(test_dir, f), f) for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]
print(len(test_imgs))

4977


In [35]:
test_faces_dir = '/home/bridge/c/nvijayas/ECE50024/test_faces'
os.makedirs(test_faces_dir, exist_ok=True)

files_not_processed = process_test(test_imgs, test_faces_dir, 'pTest')

pTest: 1000


/home/bridge/c/nvijayas/miniconda3/envs/env/lib/python3.10/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


pTest: 2000
pTest: 3000
pTest: 4000
PID pTest	NoF 468	MF 722	NL 0	SFE 0


In [38]:
x = os.listdir(test_faces_dir)
print(len(x))

3787


In [ ]:
exist = 0
no_exist = []
test_faces_dir = '/home/bridge/c/nvijayas/ECE50024/test_faces'

model_infer = YOLO('/home/bridge/c/nvijayas/ECE50024/runs/classify/train8/weights/last.pt')

to_csv = []
remaining_images = []

for i in range(4977):

    if (i) % 500 == 0:
      print(f"ITERATION ===============> {i}")
    
    img_name = f"{i}.jpg"
    img_path = os.path.join(test_faces_dir, img_name)

    if os.path.exists(img_path):
        result = model_infer(img_path, device=[7])
        r = result[0]
        x = r.probs
        x_numpy = x.data.cpu().numpy()
        max_index = np.argmax(x_numpy)
        names = r.names
        pred = names[max_index].replace('_', ' ')
        to_csv.append((i, pred))
    
    else:
        remaining_images.append(i)
        

In [4]:
print(len(remaining_images))

1190


In [5]:
test_dir = '/home/bridge/c/nvijayas/ECE50024/test'

to_csv2 = []
still_err = []

# This code was referenced
# https://pyimagesearch.com/2021/04/19/face-detection-with-dlib-hog-and-cnn/
detector = dlib.get_frontal_face_detector()
noFace = []
multiFace = []
singleFace = []

for remaining_image in remaining_images:
    img_path = os.path.join(test_dir, f"{remaining_image}.jpg")
    
    # Try to open the image normally
    try:
        pil_image = Image.open(img_path).convert('RGB')  # filePath is the path to your image file
        img = np.array(pil_image)
        detected_faces = detector(img, 1)

        if len(detected_faces) == 0:
            noFace.append(img_path)
            continue
        elif len(detected_faces) > 1:
            multiFace.append(img_path)
            continue
        else:
            singleFace.append(img_path)
            
            result = model_infer(img_path, device=[7], imgsz=64)
            r = result[0]
            x = r.probs
            x_numpy = x.data.cpu().numpy()
            max_index = np.argmax(x_numpy)
            names = r.names
            pred = names[max_index].replace('_', ' ')
            to_csv2.append((i, pred))
    except:
        still_err.append(remaining_image)

In [6]:
print(len(noFace), len(multiFace), len(singleFace))

468 722 0


In [7]:
with open('noFace.pkl', 'wb') as f:
    pickle.dump(noFace, f)

with open('multiFace.pkl', 'wb') as f:
    pickle.dump(multiFace, f)

with open('singleFace.pkl', 'wb') as f:
    pickle.dump(singleFace, f)

In [8]:
with open('noFace.pkl', 'rb') as f:
    noFace = pickle.load(f)

with open('multiFace.pkl', 'rb') as f:
    multiFace = pickle.load(f)

with open('singleFace.pkl', 'rb') as f:
    singleFace = pickle.load(f)

no faces found by dlib try cv2 now

In [13]:
face_cascade = cv2.CascadeClassifier('/home/bridge/c/nvijayas/ECE50024/haarcascade_frontalface_default.xml')

noFace_cv2 = []
multiFace_cv2 = []
singleFace_cv2 = []
still_err_cv2 = []

for img_path in multiFace:
    
    # Try to open the image normally
    try:
        # Read the input image with OpenCV
        img = cv2.imread(img_path)
        # Convert the image from BGR to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Convert into grayscale for face detection
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect faces
        detected_faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        if len(detected_faces) == 0:
            noFace_cv2.append(img_path)
            continue
        elif len(detected_faces) > 1:
            multiFace_cv2.append(img_path)
            continue
        else:
            singleFace_cv2.append(img_path)
            
    except:
        still_err_cv2.append(img_path)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [15]:
print(len(noFace_cv2), len(multiFace_cv2), len(singleFace_cv2))

6 626 89


In [17]:
with open('dlib_multi_cv2_multiFace.pkl', 'wb') as f:
    pickle.dump(multiFace_cv2, f)

modified inferences

In [18]:
with open('dlib_noFace_cv2_1Face.pkl', 'rb') as f:
    dlib_noFace_cv2_1Face = pickle.load(f)

with open('dlib_multi_cv2_1Face.pkl', 'rb') as f:
    dlib_multi_cv2_1Face = pickle.load(f)

with open('dlib_multi_cv2_multiFace.pkl', 'rb') as f:
    dlib_multi_cv2_multiFace = pickle.load(f)

Trial 4 code Start

In [2]:
with open('dlib_noFace_cv2_1Face.pkl', 'rb') as f:
    dlib_noFace_cv2_1Face = pickle.load(f)

with open('noFace.pkl', 'rb') as f:
    noFace = pickle.load(f)

print(len(dlib_noFace_cv2_1Face))
print(len(noFace))

134
468


In [3]:
# Find elements in noFace not present in dlib_noFace_cv2_1Face
dlib_set = set(dlib_noFace_cv2_1Face)
remaining_noFace = [element for element in noFace if element not in dlib_set]
print(remaining_noFace)
print(len(remaining_noFace))

['/home/bridge/c/nvijayas/ECE50024/test/18.jpg', '/home/bridge/c/nvijayas/ECE50024/test/44.jpg', '/home/bridge/c/nvijayas/ECE50024/test/47.jpg', '/home/bridge/c/nvijayas/ECE50024/test/65.jpg', '/home/bridge/c/nvijayas/ECE50024/test/67.jpg', '/home/bridge/c/nvijayas/ECE50024/test/69.jpg', '/home/bridge/c/nvijayas/ECE50024/test/76.jpg', '/home/bridge/c/nvijayas/ECE50024/test/114.jpg', '/home/bridge/c/nvijayas/ECE50024/test/132.jpg', '/home/bridge/c/nvijayas/ECE50024/test/133.jpg', '/home/bridge/c/nvijayas/ECE50024/test/148.jpg', '/home/bridge/c/nvijayas/ECE50024/test/151.jpg', '/home/bridge/c/nvijayas/ECE50024/test/155.jpg', '/home/bridge/c/nvijayas/ECE50024/test/166.jpg', '/home/bridge/c/nvijayas/ECE50024/test/185.jpg', '/home/bridge/c/nvijayas/ECE50024/test/198.jpg', '/home/bridge/c/nvijayas/ECE50024/test/205.jpg', '/home/bridge/c/nvijayas/ECE50024/test/207.jpg', '/home/bridge/c/nvijayas/ECE50024/test/210.jpg', '/home/bridge/c/nvijayas/ECE50024/test/216.jpg', '/home/bridge/c/nvijayas/E

In [12]:
from facenet_pytorch import MTCNN
import torch

/home/bridge/c/nvijayas/miniconda3/envs/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Set device
device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')

# This code was referenced for MTCNN
# https://medium.com/@saranshrajput/face-detection-using-mtcnn-f3948e5d1acb
# Initialize MTCNN
mtcnn = MTCNN(keep_all=True, device=device)

err = []
multi = []
single = []
none = []

for i, image_path in enumerate(remaining_noFace):
    print(i)
    try:
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Detect faces using MTCNN
        boxes, _ = mtcnn.detect(img_rgb)

        # If faces are detected, store the image path
        if boxes is not None:
            if len(single) == 1:
                single.append(image_path)
            else:
                multi.append(image_path)
        else:
            none.append(image_path)
    except:
        err.append(image_path)
        continue

In [32]:
print(len(err), len(none), len(single), len(multi))

22 70 0 242


Trial 4 code End

In [10]:
# helper function
def get_coords_largest_area(detected_faces):
    largest_area = 0
    largest_face_coords = (0, 0, 0, 0)
    multi = len(detected_faces) > 1
    
    for (x, y, w, h) in detected_faces:
        area = w * h
        # If this face has the largest area so far, update the variables
        if area > largest_area:
            largest_area = area
            largest_face_coords = (x, y, w, h)
    
    x, y, w, h = largest_face_coords
    return x, y, w, h, multi

def get_coords_largest_area2(detected_faces):
    largest_area = 0
    largest_face_coords = (0, 0, 0, 0)
    
    multi = len(detected_faces) > 1
    
    for box in detected_faces:
        # Extract coordinates from the bounding box
        x1, y1, x2, y2 = box
        
        # Calculate width and height
        w = x2 - x1
        h = y2 - y1
        
        area = w * h
        
        # If this face has the largest area so far, update the variables
        if area > largest_area:
            largest_area = area
            largest_face_coords = (x1, y1, x2, y2)
    
    x, y, w, h = largest_face_coords
    return int(x), int(y), int(w), int(h), multi

def extract_image_number(image_path):
    print(image_path)
    base_name = os.path.basename(image_path)
    file_name_without_extension = os.path.splitext(base_name)[0]
    image_number = int(file_name_without_extension)
    return image_number


In [28]:
print(extract_image_number(dlib_multi_cv2_multiFace[0]))

/home/bridge/c/nvijayas/ECE50024/test/9.jpg
9


In [16]:
#device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device='cpu')

In [ ]:
model_infer = YOLO('/home/bridge/c/nvijayas/ECE50024/runs/classify/train/weights/best.pt')
#extra_images = dlib_noFace_cv2_1Face + dlib_multi_cv2_1Face + dlib_multi_cv2_multiFace
#extra_images = multi + none
extra_images = noFace + multiFace

final_noFace = []   # should be 0
final_multiFace = [] # should be 0
final_singleFace = []
final_still_err = []
final_tocsv = []

print(len(extra_images))
for i, img_path in enumerate(extra_images):
    try:
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #detected_faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        detected_faces, _ = mtcnn.detect(img_rgb)

        if detected_faces is None:
            print(i, 'len None')
            result = model_infer(img_rgb, imgsz = 64)
            r = result[0]
            x = r.probs
            x_numpy = x.data.cpu().numpy()
            max_index = np.argmax(x_numpy)
            names = r.names
            pred = names[max_index].replace('_', ' ')
            final_tocsv.append((extract_image_number(img_path), pred))

        elif len(detected_faces) == 0:
            print(i, 'len 0')
            final_noFace.append(img_path)
            continue            
        
        else:
            print(i, 'len Multi or 1')
            x, y, w, h, multi_hi = get_coords_largest_area2(detected_faces)
            
            if multi_hi == True:
                final_multiFace.append(img_path)
            else:
                final_singleFace.append(img_path)
            
            cropped_img = Image.fromarray(img_rgb[y:y+h, x:x+w], 'RGB').resize((64, 64))
            result = model_infer(cropped_img)
            r = result[0]
            x = r.probs
            x_numpy = x.data.cpu().numpy()
            max_index = np.argmax(x_numpy)
            names = r.names
            pred = names[max_index].replace('_', ' ')
            final_tocsv.append((extract_image_number(img_path), pred))
            
    except Exception as e:
        print(i, 'err', e)
        final_still_err.append(img_path)
        continue

In [19]:
print(len(final_noFace), len(final_multiFace), len(final_singleFace), len(final_still_err), len(final_tocsv))
print(len(final_tocsv))
print(len(to_csv))

0 758 319 23 1167


In [26]:
x = final_tocsv + to_csv

### Final - Modify Past CSV

In [ ]:
filename = '/home/bridge/c/nvijayas/ECE50024/test_output_try4.csv'
new_filename = '/home/bridge/c/nvijayas/ECE50024/test_output_try5.csv'

with open(filename, mode='r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    headers = next(reader)
    # Convert all rows into a list of dictionaries for easier manipulation
    rows = [dict(zip(headers, row)) for row in reader]

# Modify the specified rows
for id_to_modify, new_category in x:#final_tocsv:
    for row in rows:
        if int(row['ID']) == id_to_modify:
            old_category = row['Category']  # Store the old category before changing it
            row['Category'] = new_category  # Update to the new category
            # Print the change for this row
            print(f"ID {id_to_modify}, old - {old_category} --> new - {new_category}")

In [37]:
with open(new_filename, mode='w', newline='', encoding='utf-8') as new_csvfile:
    writer = csv.DictWriter(new_csvfile, fieldnames=headers)
    writer.writeheader()
    writer.writerows(rows)

### Final - Make CSV

In [63]:
to_csv_err = [(i, 'Aaron Paul') for i in still_err]
to_csv_final = to_csv + to_csv2 + to_csv_err
dict_to_csv_final = {tup[0]: tup[1] for tup in to_csv_final}

In [72]:
print(len(to_csv_err))
print(to_csv_err)

23
[(132, 'Aaron Paul'), (238, 'Aaron Paul'), (361, 'Aaron Paul'), (991, 'Aaron Paul'), (1280, 'Aaron Paul'), (1639, 'Aaron Paul'), (1735, 'Aaron Paul'), (2166, 'Aaron Paul'), (2324, 'Aaron Paul'), (2517, 'Aaron Paul'), (2651, 'Aaron Paul'), (3368, 'Aaron Paul'), (3404, 'Aaron Paul'), (3570, 'Aaron Paul'), (3663, 'Aaron Paul'), (3802, 'Aaron Paul'), (4175, 'Aaron Paul'), (4204, 'Aaron Paul'), (4342, 'Aaron Paul'), (4684, 'Aaron Paul'), (4826, 'Aaron Paul'), (4847, 'Aaron Paul'), (4862, 'Aaron Paul')]


In [66]:
filename = "/home/bridge/c/nvijayas/ECE50024/test_output_try2.csv"

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "Category"])

    for i in range(4977):
        category = dict_to_csv_final.get(i, 'Aaron Paul')
        writer.writerow([i, category])